In [3]:
import numpy as np
import pickle
from dataset import *
from minetest_adapter import MinetestAdapter
from random import choices
import pandas as pd
from time import sleep

# Structures

In [ ]:
with open('training_data.pkl','rb') as f:
    trainingData = pickle.load(f)
    
with open('validation_data.pkl','rb') as f:
    validationData = pickle.load(f)

data = []
for i in range(len(trainingData)):
    data.append((trainingData[i][1], trainingData[i][2]))
for i in range(len(validationData)):
    data.append((validationData[i][1], validationData[i][2]))

In [ ]:
# testing that everything works as expected
a = [[[1,1,2], [3,4,5], [6,7,8]]]
b = ['nothing', 'roof', 'windows', 'windows', 'windows', 'windows', 'windows', 'none', 'wall' ]

reversed_annotation = b[1:] # exclude 'nothing' from re-indexing
reversed_annotation.reverse()
true_annotation = ['nothing']
true_annotation += reversed_annotation

print(cleanAndReindexSegmentation(np.array(a), b))
print(true_annotation)

[[[8 8 0]
  [0 0 0]
  [0 0 1]]]
['nothing', 'wall', 'none', 'windows', 'windows', 'windows', 'windows', 'windows', 'roof']


In [ ]:
structures = []
for (segmentation, annotation) in data:
    structure = cleanAndReindexSegmentation(segmentation, annotation)
    reversed_annotation = annotation[1:] # exclude 'nothing' tag from reversal
    reversed_annotation.reverse()
    true_annotation = ['nothing']
    true_annotation += reversed_annotation
    structures.append((structure, true_annotation))
    for n in range(1, 4):
        structures.append((np.rot90(structure, k=n, axes=(0, 2)), true_annotation))

In [ ]:
with open('structures.pkl', 'wb') as f:
    pickle.dump(structures, f, protocol=pickle.HIGHEST_PROTOCOL)

# Markov Chains

In [ ]:
with open('structures.pkl','rb') as f:
    structures = pickle.load(f)

In [ ]:
transitionTable, segments = calculateMarkovTransitions(structures)

In [ ]:
with open('transition_table.pkl', 'wb') as f:
    pickle.dump(transitionTable, f, protocol=pickle.HIGHEST_PROTOCOL)

with open('segments_dict.pkl', 'wb') as f:
    pickle.dump(segments, f, protocol=pickle.HIGHEST_PROTOCOL)

# Data Generation

In [4]:
with open('transition_table.pkl','rb') as f:
    transitionsTable = pickle.load(f)

with open('segments_dict.pkl','rb') as f:
    segmentsDict = pickle.load(f)

In [5]:
DIM = 24
down = ['floor', 'base', 'bottom', 'ground', 'foundation']
up = ['roof', 'top', 'layer', 'ceiling', 'ledge', 'overhang', 'platform']

def getYPositioning(structure, x, z, segment, name):
    s0, s1, s2 = segment.shape

    for word in down:
        # floor types go to the bottom
        if name.find(word) != -1:
            return 0, s1

    for word in up:
        # roof types go to the top
        if name.find(word) != -1:
            # roofs must not float
            # find highest point in designated area
            nonZeroIds = np.nonzero(
                # slice the requested area
                structure[x:x+s0, 0:DIM, z:z+s2])
            # use the highest non-zero y index
            if len(nonZeroIds[1]) == 0:
                # roof part falls to the ground
                return 1, 1+s1
            # prevent building from clipping OOB vertically
            h = nonZeroIds[1].max()
            safetyClip = h + s1 - DIM if h + s1 >= DIM else 0
            return (h - safetyClip, h + s1 - safetyClip)

    # wall types stand upright on the floor
    return 1, 1+s1

def generateStructure(annotation, segmentsDict):
    structure = np.zeros(shape=(DIM,DIM,DIM), dtype=int)
    # init building origin somewhat randomly
    [x, z] = choices(range(int(DIM/4)), k=2)
    y = 0 # vertical dim in minetest
    xDir = 1
    zDir = 1
    for segmentIndex, segmentName in enumerate(annotation):
        if segmentIndex == 0 or segmentName == "Done":
            continue
        segment = choices(segmentsDict[segmentName])[0]
        segment[segment == 1] = segmentIndex
        s0, s1, s2 = segment.shape

        # in the outer thirds of the space we invert horizontal directions
        turnPositiveZone = list(range(int(DIM/3)))
        turnNegativeZone = [2 * int(DIM/3) + i for i in range(int(DIM/3))]
        xDir = 1 if x in turnPositiveZone else xDir
        yDir = 1 if y in turnPositiveZone else yDir
        xDir = -1 if x in turnNegativeZone else xDir
        yDir = -1 if y in turnNegativeZone else yDir
        # some safety constraints
        xDir *= -1 if x+xDir*s0 >= DIM or x+xDir*s0 < 0 else 1
        zDir *= -1 if z+zDir*s2 >= DIM or z+zDir*s2 < 0 else 1
        nextX = x+xDir*s0
        nextZ = z+zDir*s2

        # update vertical position according to segment type
        y, nextY = getYPositioning(structure, x, z, segment, segmentName)

        try:
            structure[x:nextX:xDir, y:nextY, z:nextZ:zDir] = segment
        except Exception as e:
            print(e)
            print('ids:', x, nextX, xDir, '|', y, nextY, '|', z, nextZ, zDir)
            print(structure[x:nextX:xDir, y:nextY, z:nextZ:zDir].shape)

        # build along the same axis
        if s0 > s2:
            x = nextX
        else:
            z = nextZ
    return structure

mA = generateAnnotation(transitionsTable)
print(mA)
st = generateStructure(mA, segmentsDict)

['Start', 'floor', 'floor', 'roof', 'beam', 'ledge', 'ledge', 'ledge', 'ledge', 'ledge', 'ledge', 'overhang', 'wall', 'wall', 'wall', 'wall', 'wall', 'wall', 'Done']
Miney not connected


In [6]:
from tqdm import tqdm
artificial_structures = []
for i in tqdm(range(500000)):
    a = generateAnnotation(transitionsTable)
    artificial_structures.append((generateStructure(a, segmentsDict), a))

100%|██████████| 500000/500000 [11:18<00:00, 736.62it/s]


In [8]:
with open('artificial_structures.pkl', 'wb') as f:
    pickle.dump(artificial_structures[:100000], f, protocol=pickle.HIGHEST_PROTOCOL)